In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
HOME_DIR= 'D:/jupyter/tf1/ScooterHelmetDetector/'
import sys, os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

LOCAL_PACKAGE_DIR = os.path.abspath(HOME_DIR+'/keras-yolo3')
sys.path.append(LOCAL_PACKAGE_DIR)

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

#### import #### 


Using TensorFlow backend.


In [3]:
LOCAL_PACKAGE_DIR = os.path.abspath("./keras-yolo3/yolo3")

In [4]:
WEIGHT_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/weights.h5')
ANCHOR_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/yolo_anchors.txt')
CLASS_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/helmetnonhelmet_classes.txt')

In [5]:
from yolo import YOLO
#keras-yolo에서 image처리를 주요 PIL로 수행. 
from PIL import Image
sys.path.append(LOCAL_PACKAGE_DIR)

helmetAlarm = YOLO(model_path=WEIGHT_DIR,
            anchors_path=ANCHOR_DIR,
            classes_path=CLASS_DIR)
#_augmented1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
D:\jupyter\tf1\ScooterHelmetDetector\keras-yolo3\yolo3\weights.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
TEST_DIR='D:/jupyter/tf1/HelmetAlarm/data/TESTIMAGE'

In [7]:
# img = Image.open(os.path.join(TEST_DIR, 'test19.jpg'))
# detected_img = helmetAlarm.detect_image(img)
# plt.figure(figsize=(8, 8))
# plt.imshow(detected_img)

In [8]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys
import cv2
import numpy as np
import time
import os
import pyautogui

recent_prt_sec=-1
recent_prt_min=-1
location='on Screen'

class ScooterHelmetDetector(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("ScooterHelmetDetector_demo")
        self.setGeometry(150,150,10+800+10+600+15,580)
        self.initUI()

    def initUI(self):
        self.cpt = cv2.VideoCapture(0)
        self.fps=24
        
        _,self.img_o = self.cpt.read()
        self.img_o = cv2.cvtColor(self.img_o,cv2.COLOR_RGB2GRAY)
        cv2.imwrite('img_o.jpg',self.img_o)

        self.cnt=0

        self.frame = QLabel(self)
        self.frame.resize(800,500)
        self.frame.setScaledContents(True)
        self.frame.move(10,10) 

        self.movielabel=QLabel(self)
        self.movielabel.move(5+200,5+125)
        self.movielabel.resize(400,250)
        self.movielabel.setScaledContents(True)
        self.movie = QMovie('logo.gif', QByteArray(), self)
        self.movie.setCacheMode(QMovie.CacheAll)
        self.movielabel.setMovie(self.movie)
        self.movie.start()
        
        
        self.logtitle=QLabel(self)
        self.logtitle.resize(200,25)
        self.logtitle.move(10+800+15,10)
        self.logtitle.setText("탐지 내역")
        self.logtitle.setFont(QFont("맑은 고딕",10,QFont.Bold))
        

        
        self.table=QTableWidget(self)
        self.table.resize(600,520)
        self.table.move(10+800+15,10+25+5)
        self.table.setRowCount(0)
        self.table.setColumnCount(5)
        self.table.setHorizontalHeaderLabels(['날짜','시각','장소','결과','세부 정보'])
        self.table.setColumnWidth(0,self.table.width()*1/10)
        self.table.setColumnWidth(1,self.table.width()*1/10)
        self.table.setColumnWidth(2,self.table.width()*2/10)
        self.table.setColumnWidth(3,self.table.width()*3/10)
        self.table.setColumnWidth(4,self.table.width()*3/10)
        
        
    

        self.btn_on = QPushButton("화면 연결",self)
        self.btn_on.resize(100,25)
        self.btn_on.move(10+100+5,10+500+15)
        self.btn_on.setStyleSheet("background-color:#8FFFF7;" "font-weight: bold;")
        self.btn_on.clicked.connect(self.start)
        
        self.btn_on2 = QPushButton("카메라 연결",self)
        self.btn_on2.resize(100,25)
        self.btn_on2.move(10,10+500+15)
        self.btn_on2.setStyleSheet("background-color:#8FFFF7;"  "font-weight: bold;")
        self.btn_on2.clicked.connect(self.start2)
        
        self.btn_off = QPushButton("중지",self)
        self.btn_off.resize(100,25)
        self.btn_off.move(10+100+5+100+5,10+500+15)
        self.btn_off.setStyleSheet("background-color:#8FFFF7;"  "font-weight: bold;")
        self.btn_off.clicked.connect(self.stop)

        
        self.msg=QLabel(self)
        self.msg.setText("상태 메세지: ")
        self.msg.resize(100,25)
        self.msg.move(10+100+5+100+5+100+15+100+15+15,10+500+15)
        
        self.prt=QLabel(self)
        self.prt.resize(200,25)
        self.prt.move(10+100+5+100+5+100+15+100+15+25+100-20,10+500+15)
        self.prt.setText("대기 중입니다.")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : Green")
        

        self.sldr=QSlider(Qt.Horizontal,self)
        self.sldr.resize(100,25)
        self.sldr.move(10+100+5+100+5+100+15,10+500+15)
        self.sldr.setMinimum(1)
        self.sldr.setMaximum(30)
        self.sldr.setValue(24)
        self.sldr.valueChanged.connect(self.setFps)
        
        self.sldrtxt1=QLabel(self)
        self.sldrtxt1.setText("FPS  : ")
        self.sldrtxt1.resize(100,25)
        self.sldrtxt1.move(10+100+5+100+5+100+35,10+500+10+25+5)
        self.fpstxt=QLabel(self)
        self.fpstxt.move(10+100+5+100+5+100+35+35+10,10+500+10+25+12)
        self.fpstxt.setText(str(self.fps))

        

        
        self.show()

    def setFps(self):
        self.fps = self.sldr.value()
        self.prt.setText("FPS가 "+str(self.fps)+"로 조정되었습니다.")
        self.prt.setStyleSheet("Color : Black")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.fpstxt.setText(str(self.fps))
        self.timer.stop()
        self.timer.start(1000 / self.fps)

  

    def start(self):
        self.prt.setText("화면과 연결 중입니다 . . .")
        self.prt.setStyleSheet("Color : Red")
        global location
        location='on Screen'
        self.movielabel.hide()
        self.codec = cv2.VideoWriter_fourcc(*'XVID')
        self.timer=QTimer()
        self.timer.timeout.connect(self.nextFrameSlot)
        self.timer.start(int(1000/self.fps))
        
        
    def start2(self):
        self.prt.setText("카메라와 연결 중입니다 . . .")
        self.prt.setStyleSheet("Color : Red")
        global location
        location='서울시 도봉구 창5동'
        self.movielabel.hide()
        self.codec = cv2.VideoWriter_fourcc(*'XVID')
        self.timer=QTimer()
        self.timer.timeout.connect(self.nextFrameSlot2)
        self.timer.start(int(1000/self.fps))
        
    def stop(self):
        self.prt.setText("대기 중입니다.")
        self.prt.setStyleSheet("Color : Green")
        self.movielabel.show()
        self.frame.setPixmap(QPixmap.fromImage(QImage()))
        #self.frame.setPixmap(self.icon)
        self.movielabel.setMovie(self.movie)
        self.movie.start()
        self.timer.stop()

    def nextFrameSlot(self):
        self.prt.setText("탐지 중입니다 . . .")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : blue")
        pic = pyautogui.screenshot(region=(0, 0, 1920, 1080))
        cam = np.array(pic)
        img=Image.fromarray(cam)
        detected_image,label=helmetAlarm.detect_image(img)
        result=np.asarray(detected_image)
        cam=result
        img=QImage(cam,cam.shape[1], cam.shape[0],QImage.Format_RGB888) #input, width, heigth, format
        pix=QPixmap.fromImage(img)
        self.frame.setPixmap(pix)
        t=time.localtime()
        if t.tm_sec%3==0:
            self.prt.setText("탐지 중입니다 . ")
        elif t.tm_sec%3==1:
            self.prt.setText("탐지 중입니다 . . ")
        else:
            self.prt.setText("탐지 중입니다 . . .")
        
        if(not(t.tm_sec==recent_prt_sec and t.tm_min == recent_prt_min) and len(label)!=0):
            self.addRow(label)
            
    def nextFrameSlot2(self):
        self.prt.setText("탐지 중입니다 . . .")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : blue")
        _,cam=self.cpt.read() 
        cam=cv2.cvtColor(cam,cv2.COLOR_BGR2RGB) #bgr rgb로 변환
        img=Image.fromarray(cam)
        detected_image,label=helmetAlarm.detect_image(img)
        result=np.asarray(detected_image)
        cam=result
        img=QImage(cam,cam.shape[1], cam.shape[0],QImage.Format_RGB888) #input, width, heigth, format
        pix=QPixmap.fromImage(img)
        self.frame.setPixmap(pix)
        t=time.localtime()
        if t.tm_sec%3==0:
            self.prt.setText("탐지 중입니다 . ")
        elif t.tm_sec%3==1:
            self.prt.setText("탐지 중입니다 . . ")
        else:
            self.prt.setText("탐지 중입니다 . . .")
            
        
        if(not(t.tm_sec==recent_prt_sec and t.tm_min == recent_prt_min) and len(label)!=0):
            self.addRow(label)
        
    def addRow(self,label):
        rowPosition=self.table.rowCount()
        self.table.insertRow(rowPosition)
        t=time.localtime()
        self.table.setItem(rowPosition , 0, QTableWidgetItem("{}-{}-{}".format(t.tm_year, t.tm_mon, t.tm_mday)))
        self.table.setItem(rowPosition , 1, QTableWidgetItem("{}:{}:{}".format(t.tm_hour,t.tm_min,t.tm_sec)))
        self.table.setItem(rowPosition , 2, QTableWidgetItem(location))
        res=""
        detail=""
        if 'helmet' in label:
            res+="착용 : {}건      ".format(len(label['helmet']))
            detail+="helmet {}".format(str(label['helmet']))
        if 'nonhelmet' in label:
            res+="미착용 : {}건  ".format(len(label['nonhelmet']))
            detail+="nonhelmet {}".format(str(label['nonhelmet']))
        self.table.setItem(rowPosition , 3, QTableWidgetItem(res))
        self.table.setItem(rowPosition , 4, QTableWidgetItem(detail))
        self.table.scrollToBottom()
        
        global recent_prt_sec
        recent_prt_sec=t.tm_sec
        global recent_prt_min
        recent_prt_min=t.tm_min
        print(recent_prt_sec,recent_prt_min)


In [ ]:
app=QApplication(sys.argv)
w=ScooterHelmetDetector()
sys.exit(app.exec_())

(416, 416, 3)
Found 0 boxes for img
2.3149341000000163
(416, 416, 3)
Found 0 boxes for img
0.030000000000001137
(416, 416, 3)
Found 0 boxes for img
0.029827000000011594
(416, 416, 3)
Found 0 boxes for img
0.029577600000010307
(416, 416, 3)
Found 0 boxes for img
0.03208180000001448
(416, 416, 3)
Found 0 boxes for img
0.036580000000014934
(416, 416, 3)
Found 0 boxes for img
0.029901399999999967
(416, 416, 3)
Found 0 boxes for img
0.03010240000000408
(416, 416, 3)
Found 0 boxes for img
0.029810999999995147
(416, 416, 3)
Found 0 boxes for img
0.030107799999996132
(416, 416, 3)
Found 0 boxes for img
0.030110199999995757
(416, 416, 3)
Found 0 boxes for img
0.032281399999988025
(416, 416, 3)
Found 0 boxes for img
0.030248499999999012
(416, 416, 3)
Found 0 boxes for img
0.03350620000000504
(416, 416, 3)
Found 0 boxes for img
0.030447100000003502
(416, 416, 3)
Found 0 boxes for img
0.03579339999998865
(416, 416, 3)
Found 0 boxes for img
0.03055190000000607
(416, 416, 3)
Found 0 boxes for img
0.

0.03303750000000605
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.86 (375, 138) (481, 319)
{'nonhelmet': [[(375, 138), (481, 319)]]}
0.03926309999999944
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.89 (374, 139) (483, 320)
{'nonhelmet': [[(374, 139), (483, 320)]]}
0.03236069999999813
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.78 (374, 144) (486, 321)
{'nonhelmet': [[(374, 144), (486, 321)]]}
0.03246070000000145
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.76 (373, 143) (490, 322)
{'nonhelmet': [[(373, 143), (490, 322)]]}
0.0327321999999981
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.82 (375, 142) (490, 326)
{'nonhelmet': [[(375, 142), (490, 326)]]}
0.0320590999999979
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.83 (376, 145) (489, 325)
{'nonhelmet': [[(376, 145), (489, 325)]]}
0.032553499999977475
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.72 (374, 149) (491, 324)
{'nonhelmet': [[(374, 149), (491, 324)]]}
0.03215439999999603
(416, 416, 3)
Found 1 boxes for img


Found 1 boxes for img
nonhelmet 0.66 (394, 187) (519, 353)
{'nonhelmet': [[(394, 187), (519, 353)]]}
0.0468665999999871
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.78 (397, 182) (515, 359)
{'nonhelmet': [[(397, 182), (515, 359)]]}
0.03207729999999742
36 46
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.77 (398, 181) (514, 361)
{'nonhelmet': [[(398, 181), (514, 361)]]}
0.03278410000001486
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.84 (399, 179) (514, 363)
{'nonhelmet': [[(399, 179), (514, 363)]]}
0.03255219999999781
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.83 (398, 179) (516, 363)
{'nonhelmet': [[(398, 179), (516, 363)]]}
0.03817000000000803
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.81 (398, 180) (516, 363)
{'nonhelmet': [[(398, 180), (516, 363)]]}
0.03281160000000227
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.74 (399, 180) (517, 363)
{'nonhelmet': [[(399, 180), (517, 363)]]}
0.03179029999998306
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.75 (398, 179) (5

(416, 416, 3)
Found 0 boxes for img
0.03018209999999044
(416, 416, 3)
Found 0 boxes for img
0.03038739999999507
(416, 416, 3)
Found 0 boxes for img
0.03003769999997985
(416, 416, 3)
Found 0 boxes for img
0.03004780000000551
(416, 416, 3)
Found 0 boxes for img
0.03084599999999682
(416, 416, 3)
Found 0 boxes for img
0.030143400000014253
(416, 416, 3)
Found 0 boxes for img
0.030174500000015314
(416, 416, 3)
Found 0 boxes for img
0.03043859999999654
(416, 416, 3)
Found 0 boxes for img
0.030296099999986836
(416, 416, 3)
Found 0 boxes for img
0.030249200000014298
(416, 416, 3)
Found 0 boxes for img
0.04260179999999991
(416, 416, 3)
Found 0 boxes for img
0.03035469999997531
(416, 416, 3)
Found 0 boxes for img
0.03003879999999981
(416, 416, 3)
Found 0 boxes for img
0.030141200000002755
(416, 416, 3)
Found 0 boxes for img
0.030141200000002755
(416, 416, 3)
Found 0 boxes for img
0.03684559999999237
(416, 416, 3)
Found 0 boxes for img
0.03235900000001379
(416, 416, 3)
Found 0 boxes for img
0.0311

(416, 416, 3)
Found 0 boxes for img
0.03047449999999685
(416, 416, 3)
Found 0 boxes for img
0.030289000000010446
(416, 416, 3)
Found 0 boxes for img
0.030151700000004666
(416, 416, 3)
Found 0 boxes for img
0.03005779999998026
(416, 416, 3)
Found 0 boxes for img
0.03144520000000739
(416, 416, 3)
Found 0 boxes for img
0.03037100000000237
(416, 416, 3)
Found 0 boxes for img
0.03787140000000022
(416, 416, 3)
Found 0 boxes for img
0.030577199999981985
(416, 416, 3)
Found 0 boxes for img
0.030299200000001747
(416, 416, 3)
Found 0 boxes for img
0.030267500000007885
(416, 416, 3)
Found 0 boxes for img
0.030214599999993652
(416, 416, 3)
Found 0 boxes for img
0.029868600000014567
(416, 416, 3)
Found 0 boxes for img
0.029587900000024092
(416, 416, 3)
Found 0 boxes for img
0.031090199999994184
(416, 416, 3)
Found 0 boxes for img
0.029685700000015913
(416, 416, 3)
Found 0 boxes for img
0.02996190000001775
(416, 416, 3)
Found 0 boxes for img
0.030072899999993297
(416, 416, 3)
Found 0 boxes for img
0

Found 0 boxes for img
0.03217670000000794
(416, 416, 3)
Found 0 boxes for img
0.03008879999998726
(416, 416, 3)
Found 0 boxes for img
0.030300000000011096
(416, 416, 3)
Found 0 boxes for img
0.030288399999989224
(416, 416, 3)
Found 0 boxes for img
0.03147320000002196
(416, 416, 3)
Found 0 boxes for img
0.031052899999991723
(416, 416, 3)
Found 0 boxes for img
0.029601299999995945
(416, 416, 3)
Found 0 boxes for img
0.030119400000018004
(416, 416, 3)
Found 0 boxes for img
0.030027100000012297
(416, 416, 3)
Found 0 boxes for img
0.03161249999999427
(416, 416, 3)
Found 0 boxes for img
0.030196300000000065
(416, 416, 3)
Found 0 boxes for img
0.030328800000006595
(416, 416, 3)
Found 0 boxes for img
0.030046900000002097
(416, 416, 3)
Found 0 boxes for img
0.030332400000020243
(416, 416, 3)
Found 0 boxes for img
0.03120789999999829
(416, 416, 3)
Found 0 boxes for img
0.046677600000009534
(416, 416, 3)
Found 0 boxes for img
0.030344399999989946
(416, 416, 3)
Found 0 boxes for img
0.031128300000

(416, 416, 3)
Found 0 boxes for img
0.030905700000005254
(416, 416, 3)
Found 0 boxes for img
0.045516999999989594
(416, 416, 3)
Found 0 boxes for img
0.03797699999998372
(416, 416, 3)
Found 0 boxes for img
0.031314000000008946
(416, 416, 3)
Found 0 boxes for img
0.03201020000000199
(416, 416, 3)
Found 0 boxes for img
0.0314811000000077
(416, 416, 3)
Found 0 boxes for img
0.03072870000002581
(416, 416, 3)
Found 0 boxes for img
0.03161310000001549
(416, 416, 3)
Found 0 boxes for img
0.037582799999995586
(416, 416, 3)
Found 0 boxes for img
0.03128509999999096
(416, 416, 3)
Found 0 boxes for img
0.03924589999999739
(416, 416, 3)
Found 0 boxes for img
0.045591000000001713
(416, 416, 3)
Found 0 boxes for img
0.046522799999991094
(416, 416, 3)
Found 0 boxes for img
0.03151650000000927
(416, 416, 3)
Found 0 boxes for img
0.03148279999999204
(416, 416, 3)
Found 0 boxes for img
0.03183970000000613
(416, 416, 3)
Found 0 boxes for img
0.03680650000001151
(416, 416, 3)
Found 1 boxes for img
0.03947